In [3]:
# run_final_experiment.py
import os
import numpy as np
import pandas as pd
import torch
import evaluate
import demoji
from datasets import Dataset
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    EarlyStoppingCallback
)

# --- CONFIGURATION SECTION ---

# 1. Define the models for the bake-off
MODELS_TO_TEST = [
    "vinai/bertweet-base",
    "cardiffnlp/twitter-roberta-base-emotion"
]

# 2. Define the hyperparameter search space for the winning model
HYPERPARAMETER_SEARCH_SPACE = [
    {"learning_rate": 2e-5, "num_train_epochs": 5, "per_device_train_batch_size": 16},
    {"learning_rate": 3e-5, "num_train_epochs": 8, "per_device_train_batch_size": 16},
    {"learning_rate": 5e-5, "num_train_epochs": 8, "per_device_train_batch_size": 16},
    {"learning_rate": 3e-5, "num_train_epochs": 8, "per_device_train_batch_size": 32}, # Try if GPU has enough memory
]

# 3. Define raw data files
FRIENDS_DATA_PATH = '../data/data1.xlsx' # Your 1K high-quality data
KAGGLE_DATA_PATHS = ['../data/data2.xlsx', '../data/data3.xlsx'] # The generic data

# --- STEP 1: ADVANCED DATA PREPARATION ---

# --- STEP 1: ADVANCED DATA PREPARATION ---

print("--- STEP 1: Preparing Datasets from Excel Files ---")

# Load and combine data
df_friends = pd.read_excel(FRIENDS_DATA_PATH)
df_kaggle_list = [pd.read_excel(p) for p in KAGGLE_DATA_PATHS]
df_kaggle = pd.concat(df_kaggle_list, ignore_index=True)

# --- NEW: Standardize and Rename Columns ---
# First, make all column names lowercase and remove leading/trailing spaces
df_friends.columns = [col.strip().lower() for col in df_friends.columns]
df_kaggle.columns = [col.strip().lower() for col in df_kaggle.columns]

# Now, rename the 'entry' column to 'text' so the rest of the script works
# We will check if the 'entry' column exists before trying to rename it
if 'entry' in df_friends.columns:
    df_friends.rename(columns={'entry': 'text'}, inplace=True)
if 'entry' in df_kaggle.columns:
    df_kaggle.rename(columns={'entry': 'text'}, inplace=True)

print("Standardized Columns in Friends Data:", df_friends.columns)
print("Standardized Columns in Kaggle Data:", df_kaggle.columns)
# -------------------------------------------

# Clean and drop duplicates (This will now work)
df_friends.dropna(subset=['text', 'emotion'], inplace=True)
df_kaggle.dropna(subset=['text', 'emotion'], inplace=True)
df_friends.drop_duplicates(subset=['text'], inplace=True)
df_kaggle.drop_duplicates(subset=['text'], inplace=True)

# Isolate a "golden" test set from your friends' data
test_df = df_friends.sample(frac=0.2, random_state=42)

# The rest of the data forms the training pool
train_pool_df = pd.concat([df_friends.drop(test_df.index), df_kaggle], ignore_index=True)

# Split the pool into training and validation sets
train_df, val_df = train_test_split(train_pool_df, test_size=0.1, random_state=42, stratify=train_pool_df['emotion'])

print(f"Train set size: {len(train_df)}")
print(f"Validation set size: {len(val_df)}")
print(f"Test set (friends only): {len(test_df)}")

# Convert pandas DataFrames to Hugging Face Datasets
train_ds = Dataset.from_pandas(train_df)
val_ds = Dataset.from_pandas(val_df)
test_ds = Dataset.from_pandas(test_df)

# --- STEP 2: ENHANCED PREPROCESSING & CLASS WEIGHTS ---

print("\n--- STEP 2: Setting up Preprocessing and Class Weights ---")

# Initialize demoji for emoji translation
demoji.download_codes()

def preprocess_text(batch):
    # Convert emojis to text descriptions
    batch['text'] = [demoji.replace_with_desc(str(text), sep=" ") for text in batch['text']]
    return batch

train_ds = train_ds.map(preprocess_text, batched=True)
val_ds = val_ds.map(preprocess_text, batched=True)
test_ds = test_ds.map(preprocess_text, batched=True)

# Create label mappings from the training data
unique_labels = train_df['emotion'].unique()
label2id = {label: i for i, label in enumerate(unique_labels)}
id2label = {i: label for i, label in enumerate(unique_labels)}
num_labels = len(unique_labels)

def encode_labels(batch):
    batch['label'] = [label2id[label] for label in batch['emotion']]
    return batch

train_ds = train_ds.map(encode_labels, batched=True)
val_ds = val_ds.map(encode_labels, batched=True)
test_ds = test_ds.map(encode_labels, batched=True)

# Calculate class weights for handling imbalance
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.array(list(label2id.keys())),
    y=train_df['emotion']
)
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float).to("cuda")

# Create a Custom Trainer to use the class weights
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        loss_fct = torch.nn.CrossEntropyLoss(weight=class_weights_tensor)
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

# Define a shared metric computation function
metric = evaluate.load("f1")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return metric.compute(predictions=preds, references=labels, average="weighted")

# --- CORE TRAINING FUNCTION ---

def train_and_evaluate(model_checkpoint, training_args_dict, run_name):
    print(f"\n{'='*50}\nSTARTING RUN: {run_name} | MODEL: {model_checkpoint}\n{'='*50}\n")
    
    tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
    def tokenize_fn(batch):
        return tokenizer(batch["text"], truncation=True, padding="max_length", max_length=128)
    
    tokenized_train_ds = train_ds.map(tokenize_fn, batched=True)
    tokenized_val_ds = val_ds.map(tokenize_fn, batched=True)
    tokenized_test_ds = test_ds.map(tokenize_fn, batched=True)
    
    model = AutoModelForSequenceClassification.from_pretrained(
        model_checkpoint,
        num_labels=num_labels,
        id2label=id2label,
        label2id=label2id,
        ignore_mismatched_sizes=True  # <-- ADD THIS LINE
    )
    
    output_dir = f"./results/{run_name}"
    training_args = TrainingArguments(output_dir=output_dir, **training_args_dict)

    trainer = CustomTrainer( # Use the CustomTrainer with class weights
        model=model,
        args=training_args,
        train_dataset=tokenized_train_ds,
        eval_dataset=tokenized_val_ds,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
    )
    
    trainer.train()
    
    # Evaluate on the sacred, friend-only test set
    test_results = trainer.evaluate(eval_dataset=tokenized_test_ds)
    print(f"\n{'*'*20} RESULTS FOR RUN: {run_name} {'*'*20}")
    print(f"Final Test F1 Score on Friends Data: {test_results['eval_f1']:.4f}")
    
    return test_results

# --- MAIN EXECUTION SCRIPT ---

if __name__ == "__main__":
    
    # --- STEP 3: The Model Bake-Off ---
    print("\n\n--- STEP 3: BASELINE MODEL BAKE-OFF ---")
    
    baseline_results = {}
    baseline_args = {
        "evaluation_strategy": "epoch",
        "save_strategy": "epoch",
        "learning_rate": 2e-5,
        "per_device_train_batch_size": 16,
        "num_train_epochs": 5,
        "weight_decay": 0.01,
        "load_best_model_at_end": True,
        "metric_for_best_model": "f1",
        "save_total_limit": 1, # Only keep the single best checkpoint
        "lr_scheduler_type": 'linear',
        "warmup_ratio": 0.1,
        "seed": 42,
    }

    for model_name in MODELS_TO_TEST:
        run_name = f"baseline_{model_name.replace('/', '_')}"
        results = train_and_evaluate(model_name, baseline_args, run_name)
        baseline_results[model_name] = results['eval_f1']

    winning_model = max(baseline_results, key=baseline_results.get)
    print(f"\n\n--- BAKE-OFF COMPLETE ---\nScores: {baseline_results}\nWINNING MODEL: {winning_model}\n\n")

    # --- STEP 4: Hyperparameter Tuning the Winner ---
    print(f"--- STEP 4: HYPERPARAMETER TUNING FOR {winning_model} ---")
    
    tuning_results = []
    
    for i, params in enumerate(HYPERPARAMETER_SEARCH_SPACE):
        run_args = baseline_args.copy()
        run_args.update(params)
        
        run_name = f"tuning_{winning_model.replace('/', '_')}_run_{i+1}"
        results = train_and_evaluate(winning_model, run_args, run_name)
        
        tuning_results.append({
            "run_name": run_name,
            "params": params,
            "f1_score": results['eval_f1']
        })

    best_run = max(tuning_results, key=lambda x: x['f1_score'])
    
    print(f"\n\n{'='*60}\n--- EXPERIMENT COMPLETE ---\n")
    print(f"Best Run: {best_run['run_name']}")
    print(f"Best Hyperparameters: {best_run['params']}")
    print(f"Best Test F1 Score on Friends Data: {best_run['f1_score']:.4f}")
    print(f"\nYour final, best model is saved in: ./results/{best_run['run_name']}")
    print(f"{'='*60}")

--- STEP 1: Preparing Datasets from Excel Files ---
Standardized Columns in Friends Data: Index(['text', 'emotion'], dtype='object')
Standardized Columns in Kaggle Data: Index(['text', 'emotion'], dtype='object')
Train set size: 2509
Validation set size: 279
Test set (friends only): 200


C:\Users\das99\AppData\Local\Temp\ipykernel_22744\92992369.py:95: FutureWarning: The demoji.download_codes attribute is deprecated and will be removed from demoji in a future version. It is an unused attribute as emoji codes are now distributed directly with the demoji package.
  demoji.download_codes()



--- STEP 2: Setting up Preprocessing and Class Weights ---


Map: 100%|██████████| 200/200 [00:00<00:00, 64542.65 examples/s]




--- STEP 3: BASELINE MODEL BAKE-OFF ---

STARTING RUN: baseline_vinai_bertweet-base | MODEL: vinai/bertweet-base



...

{'eval_loss': 1.0504496097564697, 'eval_f1': 0.7495195699324888, 'eval_runtime': 1.4657, 'eval_samples_per_second': 190.357, 'eval_steps_per_second': 23.88, 'epoch': 8.0}


100%|██████████| 632/632 [05:05<00:00,  2.77it/s]

{'train_runtime': 305.3529, 'train_samples_per_second': 65.734, 'train_steps_per_second': 2.07, 'train_loss': 0.5965528276902211, 'epoch': 8.0}


100%|██████████| 25/25 [00:01<00:00, 24.66it/s]


******************** RESULTS FOR RUN: tuning_cardiffnlp_twitter-roberta-base-emotion_run_4 ********************
Final Test F1 Score on Friends Data: 0.6380


--- EXPERIMENT COMPLETE ---

Best Run: tuning_cardiffnlp_twitter-roberta-base-emotion_run_3
Best Hyperparameters: {'learning_rate': 5e-05, 'num_train_epochs': 8, 'per_device_train_batch_size': 16}
Best Test F1 Score on Friends Data: 0.7352

Your final, best model is saved in: ./results/tuning_cardiffnlp_twitter-roberta-base-emotion_run_3
